# Trabalho Final
## Detecção de cláusulas potencialmente injustas em contratos de Termos de Serviço utilizando fine-tuning no LEGAL-BERT

##### **Nome:** André Luiz Moreira Dutra
##### **Matrícula:** 2019006345

#### Linl da Apresentação: https://youtu.be/igoGCZgvFng

## Motivação

Com o atual crescente avanço tecnológico, cada vez mais recursos e funcionalidades vêm se tornando acessíveis ao usuário comum por meio de aplicativos, websites e servidores. Legalmente, o oferecimento destes recursos é reconhecido como uma prestação de serviços e, com isso, requer que um contrato formal de Termos de Serviço seja firmado, com o qual todas as partes envolvidas, inclusive o usuário final, devem concordar.

No entanto, a demanda por serviços cada vez mais rápidos e de fácil acesso tem diminuido cada vez mais o interesse dos usuários em ler estes contratos e garantir que seus direitos são garantidos. Unindo isso ao crescente interesse de empresas de tecnologia na coleta e comercialização de dados pessoais dos usuários, principalmente para fins de marketing, tornou-se uma prática comum a ocultação de contratos de Termos de Serviço, camuflados por meio de links pequenos e escondidos que levam a textos muitas vezes em uma formatação pouco intuitiva, extremamente extensos e de linguagem densa, técnica e jurídica. Os contratos são omissos em favor de botões de confirmação, popularmente conhecidos como checkboxes contendo apenas o texto "Li e aceito os termos de uso", os quais muitas vezes induzem o usuário a aceitar o contrato sem lê-lo.

Com isso, apresento neste trabalho um modelo de detecção de cláusulas potencialmente injustas em contratos de Termos de Serviço relativos utilizando um modelo de classificação de sequências treinado como um fine-tuning do LEGAL-BERT, uma arquitetura BERT pré-treinada em textos jurídicos dos Estados Unidos e da União Europeia. O fine-tuning e teste do modelo foi feito utilizando uma base de cerca de 20.000 cláusulas agrupadas em 10 classes: uma representando cláusulas justas e nove representando diferentes tipos de injustiças ao usuário que as cláusulas podem apresentar. Será apresentado o tratamento dos dados, o pré-processamento, a definição do modelo utilizado e dos hiperparâmetros de treinamento e os resultados gerais e por classe obtidos.

Os pacotes utilizados neste trabalho são os seguintes:

In [1]:
import pandas as pd
from datasets import Dataset, DatasetDict

from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from numpy import argmax

import torch
from transformers import AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments

from torch.utils.data import DataLoader

2023-12-13 22:25:53.981785: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Trabalhos Relacionados

As principais abrodagens de classificação de cláusulas de Termos de Serviço por grau de injustiça foram feitas por Lippe et. al nos próprios artigos que introduzem a base utilizada neste trabalho [[1](#ref)][[2](#ref)], nos quais foram testados os modelos considerados estado-da-arte na época: SVMs, CNNs, LSTMs em 2019 e, no artigo mais recente, End-to-End Memory Networks (MANN), uma arquitetura recorrente de 2015[[5](#ref)]. Até o momento, nenhum artigo da literatura tentou aplicar modelos baseados em transformer encoders como o BERT, que são o estado-da-arte atual em classificação de sentenças, a esta tarefa. O mais próximo já feito foi um classificador de cláusulas injustas no contexto de contratos comerciais e empresariais gerado por Singhal et. al por meio de fine-tuning do BERT original, atingindo acurácia de 84% [[3](#ref)].

## Base de Dados

A base de dados utilizada foi a ToS100, uma base composta de cerca de 20.0000 cláusulas, na forma de strings, extraídas de 100 documentos dos Estados Unidos referentes às empresas de serviços de tecnologia mais relevantes em termos de relevância global, número de usuários e tempo desde que o serviço foi estabelecido. A base constitui a maior e mais completa fonte de cláusulas de Termos de Serviço, rotuladas ou não, disponível na literatura. Ela foi gerada por Lippe et. al em 2019 e atualizada em 2021, e se encontra disponível no github do artigo [[4](#ref)].

A base classifica cada cláusula como uma de nove categorias de injustiça não-excludentes:

* **CR - Remoção de Conteúdo:** A cláusula confere ao provedor o direito de modificar/excluir o conteúdo do usuário, incluindo compras dentro do aplicativo;

* **LTD - Limitação de Responsabilidade:** A cláusula estipula que o dever de pagar danos é limitado ou excluído ao provedor do serviço sob determinadas condições;

* **J - Jurisdição:** A cláusula especifica quais tribunais terão competência para julgar disputas sob o contrato. Neste caso, foram consideradas as cláusulas que estabelecem que qualquer processo judicial possa ocorrer fora do local de residência do usuário (ou seja, em uma cidade ou país diferente);

* **LAW - Escolha de Lei:** A cláusula especifica qual lei regerá o contrato, significando também qual lei será aplicada na eventual resolução de uma disputa decorrente do contrato. Neste caso, foram consideradas as cláusulas que estabelecem que o contrato será regido por leis de um país potencialmente diferente do país de residência do usuário;

* **A - Arbitragem:** A cláusula requer ou permite que as partes resolvam disputas por meio de um processo de arbitragem, antes que o caso vá para o tribunal, com algumas cláusulas permitindo que a decisão do árbitro do processo não necessariamente siga o que é definido na lei;

* **CH - Mudança Unilateral:** A cláusula permite que o provedor de serviços altere e modifique os termos de serviço e/ou o próprio serviço sob determinadas condições;

* **TER - Rescisão Unilateral:** A cláusula confere ao provedor o direito de suspender e/ou rescindir o serviço e/ou o contrato sob determinadas condições;

* **PINC - Privacidade Inclusa:** A cláusula afirma que os consumidores consentem com a política de privacidade simplesmente usando o serviço;

* **USE - Contrato por Uso:** A cláusula estipula que o consumidor está vinculado aos termos de uso do serviço simplesmente usando-o, sem a necessidade de marcar que leu e aceitou esses termos.

É importante ressaltar que as labels classificam as cláusulas com relação à sua injustiça aparente em relação ao usuário/consumidor, o que pode ser uma definição subjetiva. É notável que a maioria dos tipos de cláusulas descritos são legais e, sob condições válidas, não apresentam injustiça ao usuário. Por isso, este modelo se destina a detectar cláusulas potencialmente injustas em um contrato, ou seja, que sejam relevantes o suficiente para que sejam lidas e interpretadas pelo usuário ao fim.

O dataset original define, assim, o problema como um problema de classificação multi-label, com cada cláusula podendo possuir múltiplas labels dentre as descritas acima, e com as cláusulas justas sendo representadas por aquelas que não possuem nenhuma das labels. Observe uma visualização do dataset:

In [2]:
tos100_raw_df = pd.read_csv('ToS_100.csv')
tos100_raw_df.head()

,Unnamed: 0,A,CH,CR,J,LAW,LTD,PINC,TER,USE,document,document_ID,label,text,TER_targets,LTD_targets,A_targets,CH_targets,CR_targets
0,0,0,0,0,0,0,0,0,0,0,Mozilla,0,0,websites & communications terms of use,NaN,NaN,NaN,NaN,NaN
1,1,0,0,0,0,0,0,0,0,0,Mozilla,0,0,please read the terms of this entire document ...,NaN,NaN,NaN,NaN,NaN
2,2,0,0,0,0,0,0,0,0,1,Mozilla,0,1,by accessing or signing up to receive communic...,NaN,NaN,NaN,NaN,NaN
3,3,0,0,0,0,0,0,0,0,0,Mozilla,0,0,our websites include multiple domains such as ...,NaN,NaN,NaN,NaN,NaN
4,4,0,0,0,0,0,0,0,0,0,Mozilla,0,0,you may also recognize our websites by nicknam...,NaN,NaN,NaN,NaN,NaN


Após analisar o dataset, podemos perceber que, embora as cláusulas sejam apresentadas como um problema de classificação multilabel, apenas 1% das instâncias possuem mais de uma classe. Com isso, optei por descartar as cláusulas multilabel e transformar o problema em um problema de classificação multiclasse, com classes multuamente excludentes. Desse modo, cada cláusula injusta restante foi classificada com sua respectiva label, adicionando o prefixo "UNFAIR-" para facilitar a interpretação (por exemplo, uma cláusula com uma única label "CR" foi classificada na classe "UNFAIR-CR") e as cláusulas sem nenhuma label foram clássificadas como "FAIR" (justas):

In [3]:
labels_num = tos100_raw_df[['A','CH','CR','J','LAW','LTD','PINC','TER','USE']].sum(axis=1)
print(f'Cláusulas multilabel: {len(labels_num[labels_num > 1])}')
print(f'Clausulas totais:   {len(tos100_raw_df)}')

Cláusulas multilabel: 205
Clausulas totais:   20417


In [4]:
#Removemos as últimas colunas, sem informação relevante e com valores NaN
tos100_df = tos100_raw_df.dropna(axis=1).copy()

#Transformamos as colunas binárias de multilabels em uma coluna categória de labels.
def get_row_label(row):
    possible_labels = ['A','CH','CR','J','LAW','LTD','PINC','TER','USE']
    labels = [label for label in possible_labels if row[label] == 1]

    return 'FAIR' if len(labels) == 0 else f'UNFAIR-{labels[0]}' if len(labels) == 1 else 'MULTILABEL'

tos100_df['label'] = tos100_df.apply(get_row_label, axis=1)

#Removemos as entradas multilabel e selecionamos apenas a colunas necessárias (a empresa de origem, o texto de input e a label de output)

tos100_df = tos100_df[tos100_df['label'] != 'MULTILABEL']
tos100_df = tos100_df[['document', 'text', 'label']]
tos100_df = tos100_df.rename(columns={'document':'company'}).reset_index(drop = True)

tos100_df.head()

,company,text,label
0,Mozilla,websites & communications terms of use,FAIR
1,Mozilla,please read the terms of this entire document ...,FAIR
2,Mozilla,by accessing or signing up to receive communic...,UNFAIR-USE
3,Mozilla,our websites include multiple domains such as ...,FAIR
4,Mozilla,you may also recognize our websites by nicknam...,FAIR


## Divisão do Dataset em Treino, Validação e Teste

O dataset foi dividido em treino, validação e teste em proporções 76%, 4% e 20%, respectivamente. Esta divisão foi gerada dividindo primeiro o dataset em treino e teste em proporções 80% e 20%, e em seguida dividindo o treino em treino e validação com proporções 95% e 5%. A divisão foi feita de maneira estratificada, uma vez que o dataset é extremamente desbalanceado em favor de cláusulas justas, conforme será abordado mais à frente.

In [5]:
#Geração do dataset huggingface e separação em treino, validação e teste

raw_dataset = Dataset.from_pandas(tos100_df[['label', 'text']]).class_encode_column('label')
trainval_test = raw_dataset.train_test_split(test_size=0.2, stratify_by_column='label')
train_val = trainval_test['train'].train_test_split(test_size=0.05, stratify_by_column='label')

tos100_dataset = DatasetDict({
    'train': train_val['train'],
    'validation': train_val['test'],
    'test': trainval_test['test']
})

label_list = tos100_dataset['train'].features['label'].names
id2label = {idx: label for idx, label in enumerate(label_list)}
label2id = {label: idx for idx, label in enumerate(label_list)}

tos100_dataset['train'][0]

Casting to class labels:   0%|          | 0/20212 [00:00<?, ? examples/s]

{'label': 0,
 'text': "some particularly egregious examples of `` bad things '' are listed in this section ."}

## Modelo Utilizado e Pré-Processamento

O modelo proposto neste trabalho se trata de um fine-tuning do LEGAL-BERT para a tarefa de classificação de sequências. O LEGAL-BERT é a arquitetura BERT base da família dos LEGAL-BERTS, modelos BERT pré-treinados em textos jurídicos [[6](#ref)]. O LEGAL-BERT-base, especificamente, possui um vocabulário próprio e foi pré-treinado com textos jurídicos legais dos EUA e da União Europeia, que constituem justamente a base contextual geral do dataset de cláusulas de Termos de Serviço utilizado. 

Como o LEGAL-BERT possui um vocabulário e tokenizer próprios, seu tokenizer foi utilizado neste trabalho para pré-processar as entradas:

In [6]:
tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")

def preprocess(examples):
  return tokenizer(examples['text'], truncation=True)

tokenized_dataset = tos100_dataset.map(preprocess, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/15360 [00:00<?, ? examples/s]

Map:   0%|          | 0/809 [00:00<?, ? examples/s]

Map:   0%|          | 0/4043 [00:00<?, ? examples/s]

# Métricas de Avaliação

Para avaliar a validação e o teste, serão usadas as métricas básicas de classificação multiclasse: acurácia, precisão, revocação e f1. Como o dataset é extremamente desbalanceado em favor da classe de cláusulas justas, em relação às demais 8 classes de cláusulas injustas, a precisão, revocação e f1 serão calculadas tirando a média aritméticas entre as métricas de cada classe, como forma de tomar com a mesma importância todas as classes, mesmo que com tamanhos diferentes.

Para a métrica de avaliação classe-a-classe do teste, será utilizada a revocação, uma vez que ela mede justamente o atributo de interesse do trabalho. Como o grau de injustiça de uma cláusula depende da interpretação pessoal do usuário, é possível que mesmo as cláusulas classificadas como potencialmente injustas não sejam interpretadas como injustas pelo usuário. Com isso, existe um grau de tolerância maior a cláusulas justas classificadas como injustas (falsos negativos).

Por outro lado, não é interessante que o nosso modelo classifique como justa uma cláusula potencialmente injusta, já que este erro pode levar à aceitação de um contrato injusto por parte do usuário. Por isso, optamos pela métrica de recall, que mede justamente a porcentagem de cláusulas identificadas corretamente como pertencente à classe em relação ao total de cláusulas daquela classe. Ou seja, ao tentar maximizar o recall, desejamos que todas as cláusulas da classe sejam identificadas, mesmo que cláusulas de outras classes sejam identificadas incorretamente no processo, como o caso dos falsos-positivos para cláusulas justas.

In [7]:
def compute_metrics(p):
  prediction_logits, true_labels = p
  predicted_labels = argmax(prediction_logits, axis=1)

  return {
    "precision": precision_score(true_labels, predicted_labels, average = 'macro'),
    "recall": recall_score(true_labels, predicted_labels, average = 'macro'),
    "f1": f1_score(true_labels, predicted_labels, average = 'macro'),
    "accuracy": accuracy_score(true_labels, predicted_labels),
  }

## Hiperparâmetros do Modelo

Conforme mencionado anteriormente, o modelo base utilizado é o LEGAL-BERT. Como forma de otimizar o espaço alocado na GPU, a versão com pesos do tipo float32 foi utilizada. Como parâmetros do modelo, estou usando taxa de aprendizado 0.00002, 3 épocas, weight decay de 0.1 e tamanho de batch igual a 3. O tamanho de batch foi escolhido devido às limitações de memória, e a taxa de aprendizado, número de épocas e weight decay foram escolhidos usando como referência os minicursos do huggingface utilizando modelos baseados no BERT.

No código a seguir, temos a instanciação do modelo, definição dos hiperparâmetros e inicialização do treinador com os dados de treino e validação. Na célula seguinte, o modelo é treinado.

In [8]:
torch.cuda.empty_cache()

model = AutoModelForSequenceClassification.from_pretrained("nlpaueb/legal-bert-base-uncased", torch_dtype=torch.float32, num_labels = len(label_list), id2label = id2label, label2id = label2id)

training_args = TrainingArguments(
    output_dir="model_results",
    learning_rate=2e-5,
    per_device_train_batch_size=3,
    per_device_eval_batch_size=3,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

Some weights of the model checkpoint at nlpaueb/legal-bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification wer

In [9]:
trainer.train()
results = trainer.evaluate()
results

/home/andre/miniconda3/envs/notebook_env/lib/python3.10/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/15360 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 0.7061, 'learning_rate': 1.9348958333333336e-05, 'epoch': 0.1}
{'loss': 0.5827, 'learning_rate': 1.869791666666667e-05, 'epoch': 0.2}
{'loss': 0.4867, 'learning_rate': 1.8046875e-05, 'epoch': 0.29}
{'loss': 0.3824, 'learning_rate': 1.7395833333333334e-05, 'epoch': 0.39}
{'loss': 0.3228, 'learning_rate': 1.6744791666666668e-05, 'epoch': 0.49}
{'loss': 0.3115, 'learning_rate': 1.609375e-05, 'epoch': 0.59}
{'loss': 0.3084, 'learning_rate': 1.5442708333333335e-05, 'epoch': 0.68}
{'loss': 0.3509, 'learning_rate': 1.479166666666667e-05, 'epoch': 0.78}
{'loss': 0.289, 'learning_rate': 1.4140625000000002e-05, 'epoch': 0.88}
{'loss': 0.2736, 'learning_rate': 1.3489583333333334e-05, 'epoch': 0.98}


  0%|          | 0/270 [00:00<?, ?it/s]

...

/home/andre/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.30348947644233704, 'eval_precision': 0.656730519025601, 'eval_recall': 0.6552615193026152, 'eval_f1': 0.6521785576764337, 'eval_accuracy': 0.9517923362175525, 'eval_runtime': 8.1186, 'eval_samples_per_second': 99.648, 'eval_steps_per_second': 33.257, 'epoch': 3.0}
{'train_runtime': 2022.1693, 'train_samples_per_second': 22.787, 'train_steps_per_second': 7.596, 'train_loss': 0.24181095696985722, 'epoch': 3.0}


  0%|          | 0/270 [00:00<?, ?it/s]

/home/andre/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.2703166604042053,
 'eval_precision': 0.5985592816274635,
 'eval_recall': 0.6602729348277294,
 'eval_f1': 0.6251847789891268,
 'eval_accuracy': 0.9443757725587144,
 'eval_runtime': 8.098,
 'eval_samples_per_second': 99.901,
 'eval_steps_per_second': 33.341,
 'epoch': 3.0}

## Inferência e Teste

As métricas de validação do modelo acima sugerem resultados promissores, com acurácia de 95% e demais métricas de cerca de 70%. Vamos agora realizar a inferência e calcular as métricas de desempenho para o conjunto de teste.

O código a seguir implementa as funções de inferência e teste, e testa o modelo implementado contra a base de teste que criamos a partir do ToS100:

In [10]:
#Selecionamos o dataset de teste
test_dataset = tokenized_dataset['test']

#Selecionamos a coluna de labels verdadeiras e a removemos do dataset de inferência 
true_labels = test_dataset['label']

test_dataset = test_dataset.remove_columns(['text', 'label'])

#Realizamos a inferência em batches para todo o conjunto de testes
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=data_collator)
prediction_logits, _, _, _ = trainer.prediction_loop(test_loader, description='prediction')

/home/andre/miniconda3/envs/notebook_env/lib/python3.10/site-packages/transformers/trainer_pt_utils.py:411: FutureWarning: DistributedTensorGatherer is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


  0%|          | 0/253 [00:00<?, ?it/s]

In [11]:
#Calculamos as métricas gerais do modelo para a base de teste
predicted_labels = argmax(prediction_logits, axis=1)

print(f'Acurácia do teste:        {accuracy_score(true_labels, predicted_labels)}')
print(f'Precisão média do teste:  {precision_score(true_labels, predicted_labels, average="macro")}')
print(f'Revocação média do teste: {recall_score(true_labels, predicted_labels, average="macro")}')
print(f'F1 médio do teste:        {f1_score(true_labels, predicted_labels, average="macro")}')

Acurácia do teste:        0.9478110314123176
Precisão média do teste:  0.6838669328804864
Revocação média do teste: 0.7302754979077282
F1 médio do teste:        0.7003685388022327


/home/andre/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Resultados Gerais

Como podemos observar acima, o modelo apresentou, conforme observado na validação, desempenho de cerca de 95% para acurácia e cerca de 70% nas demais métricas. Embora 70% de precisão e principalmente recall não sejam os valores ideais no contexto de uma classificação, ainda podemos considerar os resultados como bons, principalmente levando em consideração o balanceamento da base. Conforme mencionado anteriormente, a base é extremamente desbalanceada em favor de cláusulas justas. Conforme poderá ser observado na tabela seguinte, as cláusulas justas constituem 89% de toda a base, com as demais 9 classes de cláusulas injustas constituindo os demais 11%, variando cada uma de 3% a 0.25% de todas as instâncias. Com isso, levando em consideração o desbalanceamento da base, podemos considerar o resultado como positivo.

In [12]:
#Calculamos o recall de cada classe e apresentamos, junto ao número total de intâncias da classe na base
label_recalls = recall_score(true_labels, predicted_labels, labels=range(len(label_list)), average=None)

s = pd.Series({label: precision for label, precision in zip(label_list, label_recalls)})
df = pd.DataFrame(s, columns=['recall'])

df['Nº of instances'] = tos100_df[['label', 'text']].groupby('label').count()

sorted_df = df.sort_values('recall', ascending = False)
print('Revocações totais:')
display(sorted_df)

Revocações totais:


,recall,Nº of instances
UNFAIR-J,1.000000,114
FAIR,0.968750,18239
UNFAIR-LAW,0.920000,124
UNFAIR-USE,0.875000,242
UNFAIR-TER,0.806452,308
UNFAIR-LTD,0.750000,599
UNFAIR-A,0.666667,105
UNFAIR-CH,0.660714,281
UNFAIR-CR,0.655172,146
UNFAIR-PINC,0.000000,54


## Resultados de Revocação por Classe

Na tabela acima temos a revocação (recall) da classificação de cada classe, junto ao número de instâncias da classe na base completa.

Conforme mencionado, as classes injustas possuem muito menos instâncias do que as classes justas. De maneira geral, a distribuição de instâncias afetou bastante os resultados de cada classe, com classes com menos instâncias apresentando recalls menores que classes com mais instâncias. Neste contexto, se destacam as cláusulas justas ("FAIR"), que são as mais proeminentes e apresentaram maior recall, e as de privacidade inclusa ("PINC"), que, sendo a menos presente nos dados, contendo apenas 54 instâncias (0.25%), não teve nenhum exemplo classificado corretamente, e com isso teve recall 0. Isso é especialmente preocupante, uma vez que, dentre todas as cláusulas, as cláusulas PINC foram as únicas discutidas no artigo da base de dados como claramente injustas, ao invés de potencialmente.

No entanto, algumas classes apresentaram altas revocações apesar de não estarem entre as classes contendo a maior quantidade de exemplos, sendo elas a de jurisdição ("J"), com recall total (100%) e a de escolha de lei ("LAW"). Possivelmente, o que destacou esta classe foram termos relacionados a localidade, uma vez que os critérios de injustiça de ambas dizem respeito ao local (cidade, estado, país) da lei ou jurisdição levada em consideração, e são as únicas que os têm como temas principais.

## Conclusão

A partir dos resultados obtidos, podemos concluir que, embora o modelo tenha apresentado resultados promissores, ainda podemos explorar mais formas de melhorar o seu desempenho, principalmente levando em consideração o potencial que o modelo apresenta para a tarefa de classificação de sequências em outros âmbitos. No entanto, os resultados foram majoritariamente positivos, com 3 das classes classificadas apresentando desempenho (recall) acima de 90%, 7 apresentando recall acima de 70% e 9 apresentando recall acima de 65%. Um dos principais desafios encontrados na tarefa é a definição de uma base consistente e balanceada, uma vez que existem poucas bases para a tarefa explorada e todas apresentam um desbalanceamento grande na base com relação a cláusulas justas. Por fim, com este trabalho pudemos aplicar modelos estado-da-arte em processamento de linguagem natural a tarefas relevantes para o dia-a-dia de pessoas inseridas no contexto de serviços tecnológicos com resultados consideravelmetne positivos em relação ao que já foi feito até o momento neste contexto.

## Referências<a id='ref'></a>

1. Ruggeri, F., Lagioia, F., Lippi, M. et al. (2022) [Detecting and explaining unfairness in consumer contracts through memory networks.](https://doi.org/10.1007/s10506-021-09288-2) Artif Intell Law 30, 59–92.


2. Lippi M, Pałka P, Contissa G, Lagioia F, Micklitz HW, Sartor G, Torroni P (2019) [CLAUDETTE: an automated detector of potentially unfair clauses in online terms of service.](https://arxiv.org/abs/1805.01217) Artif Intell Law 27(2):117–139

3. Ruggeri, F., Lagioia, F., Lippi, M. et al. (2022) [Memnet_ToS Repository](https://github.com/federicoruggeri/Memnet_ToS/tree/master) Github.

4. Singhal et al., (2023). [Towards Mitigating Perceived Unfairness in Contracts from a Non-Legal Stakeholder’s Perspective](https://aclanthology.org/2023.nllp-1.11) NLLP-WS 2023

5. Sukhbaatar S, Weston J, Fergus R, et al (2015) [End-to-end memory networks. In: Advances in neural information processing systems](https://arxiv.org/abs/1503.08895), pp 2440–2448

6. I. Chalkidis, M. Fergadiotis, P. Malakasiotis, N. Aletras and I. Androutsopoulos. (2020)["LEGAL-BERT: The Muppets straight out of Law School". In Findings of Empirical Methods in Natural Language Processing](https://aclanthology.org/2020.findings-emnlp.261) EMNLP 2020.